# ME5124 – Automation and Control of Manufacturing Systems  
## PLC Programming – Sample Problems & Solutions

This notebook documents **three PLC problems** (A, B1, B2) used in ME5124.  
Programs were originally implemented using Siemens s700.  
Here, they are reproduced in **plain text ladder logic** and **instruction list (IL)** for teaching and reference.

Each problem includes:

- Problem statement  
- Text-based ladder logic (ASCII diagram)  
- Instruction List (IL)  

## Problem A — Start/Stop Motor with Seal-in and Safety

### **Problem Statement**
Design a PLC ladder program to control a motor using a **START** and **STOP** push button:

1. Pressing **START** energizes the motor and keeps it latched.  
2. Pressing **STOP** immediately turns the motor OFF.  
3. STOP button is normally closed.  
4. Include a seal-in (holding) circuit.

Inputs:  
- `X0` — START push button  
- `X1` — STOP push button (NC)

Output:  
- `Y0` — MOTOR

### Ladder Logic (ASCII Diagram)

```text
// Rung 1: Motor latch
|----[  X0 START  ]----+------------------------( Y0 MOTOR )
|                      |
|    [  Y0 MOTOR  ]----+
|
|----[ /X1 STOP   ]------------------------------------------|

In [ ]:
// Instruction List (IL)
LD     X0        ; START
OR     Y0        ; Latching contact
ANDN   X1        ; STOP push-button NC
OUT    Y0        ; Output -> MOTOR

## Problem B1 — Two-Conveyor Sequencing with Sensors

### **Problem Statement**

Develop a ladder program to control **two conveyors** with **sensor feedback**:

1. When `START` is pressed, **Conveyor 1** runs until sensor `S1` detects a part.  
2. When `S1` is activated, stop Conveyor 1 and start **Conveyor 2**.  
3. Conveyor 2 runs until `S2` is activated.  
4. When `S2` turns ON, stop Conveyor 2 and turn ON a completion lamp.  
5. `STOP` immediately halts everything.

Inputs:  
- `X0` — START  
- `X1` — STOP (NC)  
- `X2` — Sensor S1  
- `X3` — Sensor S2  

Outputs:  
- `Y0` — Conveyor 1  
- `Y1` — Conveyor 2  
- `Y2` — Completion lamp  

### Ladder Logic (ASCII Diagram)

```text
// Rung 1: RUN latch
|----[  X0 START  ]----+------------------------( M0 RUN )
|                      |
|    [  M0 RUN   ]-----+
|
|----[ /X1 STOP   ]-----------------------------------------|

// Rung 2: Conveyor 1 (until S1)
|----[  M0 RUN   ]----[ /X2 S1 ]-----------------( Y0 CONV1 )

// Rung 3: Conveyor 2 (after S1, until S2)
|----[  M0 RUN   ]----[  X2 S1 ]----[ /X3 S2 ]--( Y1 CONV2 )

// Rung 4: Completion lamp
|----[  X3 S2   ]--------------------------------( Y2 LAMP )

In [ ]:
// Instruction List (IL)

// Rung 1 – RUN latch
LD     X0
OR     M0
ANDN   X1
OUT    M0

// Rung 2 – Conveyor 1
LD     M0
ANDN   X2
OUT    Y0

// Rung 3 – Conveyor 2
LD     M0
AND    X2
ANDN   X3
OUT    Y1

// Rung 4 – Completion lamp
LD     X3
OUT    Y2

## Problem B2 — Batch Tank Filling with Level Switches and Alarm

### **Problem Statement**

Create a ladder logic program for a **batch filling system** with safety features:

1. On `START`, open inlet valve and run pump until **High Level** switch is ON.  
2. When High Level is reached, stop filling and turn ON `Batch Complete` lamp.  
3. If `EMERGENCY` or `OVERFLOW` triggers, stop everything and activate alarm.  
4. A `RESET` button clears faults once conditions are safe.

Inputs:  
- `X0` — START  
- `X1` — RESET  
- `X2` — High level  
- `X3` — EMERGENCY  
- `X4` — OVERFLOW  

Outputs:  
- `Y0` — Inlet valve  
- `Y1` — Pump  
- `Y2` — Batch complete lamp  
- `Y3` — Alarm  

Internal:  
- `M0` — RUN latch

### Ladder Logic (ASCII Diagram)

```text
// Rung 1: RUN latch (no faults)
|----[  X0 START   ]----+----[ /X3 EMG ]----[ /X4 OVERFLOW ]----( M0 RUN )
|                       |
|    [  M0 RUN    ]-----+
|
|----[  X1 RESET   ]--------------------------------------------( RST )

// Rung 2: Inlet valve & pump while filling
|----[  M0 RUN    ]----[ /X2 HIGH_LEVEL ]-----------( Y0 INLET )
|----[  M0 RUN    ]----[ /X2 HIGH_LEVEL ]-----------( Y1 PUMP )

// Rung 3: Batch complete
|----[  X2 HIGH_LEVEL ]-----------------------------( Y2 COMPLETE )

// Rung 4: Alarm
|----[  X3 EMG ]------------------------------------( Y3 ALARM )
|----[  X4 OVERFLOW ]-------------------------------( Y3 ALARM )

In [ ]:
// Instruction List (IL)

// Rung 1 – RUN latch with faults
LD     X0
OR     M0
ANDN   X3
ANDN   X4
OUT    M0

// Rung 2 – Inlet valve
LD     M0
ANDN   X2
OUT    Y0

// Rung 3 – Pump
LD     M0
ANDN   X2
OUT    Y1

// Rung 4 – Batch complete
LD     X2
OUT    Y2

// Rung 5 – Alarm
LD     X3
OR     X4
OUT    Y3

## End of PLC Sample Problems

These examples demonstrate typical ladder structures used in ME5124:
- Multi-stage sequences  
- Sensor-based automation  
- Fault handling and interlocks  